# Obtain Hex Addresses

We will use reverse geocoding to find the nearest address to each hex centre.

In [2]:
import geopandas as gpd
import numpy as np

from ratelimiter import RateLimiter
from diskcache import Cache
from tqdm.notebook import tqdm

from geopy.distance import distance
from geopy import Nominatim

import plotly.express as px

In [3]:
df_hex = gpd.read_file('../data/BBMP_hex.geojson')
print(df_hex.shape)
df_hex.head()

(942, 6)


,id,ward_no,centre_lat,centre_lon,resolution,geometry
0,8860169665fffff,1,13.113131,77.601091,8,"POLYGON ((77.60526 13.11063, 77.60535 13.11561..."
1,886016975bfffff,1,13.120623,77.597002,8,"POLYGON ((77.60117 13.11812, 77.60126 13.12311..."
2,8860169751fffff,1,13.128092,77.601343,8,"POLYGON ((77.60552 13.12559, 77.60560 13.13058..."
3,8860169669fffff,1,13.098170,77.600839,8,"POLYGON ((77.60501 13.09567, 77.60510 13.10065..."
4,8860169759fffff,1,13.120601,77.605432,8,"POLYGON ((77.60961 13.11810, 77.60969 13.12308..."


## Reverse Geocoding

Data is loaded from the GeoJSON file created earlier and displayed.

We obtain the closest addresses of each hex centre using Nominatim reverse geocoding. We need to cache the responses received, since querying Nominatim is quite slow, and we want to avoid repeating requests.

In [4]:
# Set cache location for Nominatim requests
nom_cache = '../data/cache/nominatim'

# Create Nominatim geocoder object with custom user agent as required by their terms of service
geocoder = Nominatim(user_agent = 'coursera_capstone')

# Create RateLimiter object to ensure we don't exceed Nominatim's 1 reqest per second rule
limiter = RateLimiter(max_calls = 1, period = 1)

In [5]:
# Empty lists to store responses
address = []
addr_coord = []

points = df_hex[['centre_lat', 'centre_lon']].to_records()

queries = 0 # Added variable to track number of records retrieved from cache

with Cache(nom_cache) as cache:
    for p in tqdm(points):
        query = (p.centre_lat, p.centre_lon)
        key = str(query) #! key must be a unique string
        
        if key in cache:
            response = cache[key] # Read cached value
        else:
            with limiter:
                response = geocoder.reverse(query, timeout = 30, addressdetails=True)
                cache[key] = response # Set cache value
                queries += 1
            
        address.append(response.address)
        addr_coord.append((response.latitude, response.longitude))
print('{} new queries made.'.format(queries))

  0%|          | 0/942 [00:00<?, ?it/s]

0 new queries made.


## Calculating Distances

We use geopy to calculate the geodesic distance between each hexagon centre and the address obtained. We want to check if the addresses provided are close to the hex centres.

In [6]:
centre_coord = df_hex[['centre_lat', 'centre_lon']].to_records(index=False)

address_error = [] # Empty list to store data

for centre, addr in zip(centre_coord, addr_coord):
    dist = distance(centre, addr).meters
    address_error.append(dist) # Use Geopy function

# Plot histogram
address_err_fig = px.histogram(
    x = address_error,
    #// nbins = 200,
    histnorm = 'percent',
    cumulative = True,
    template = 'plotly',
)

address_err_fig.update_layout(
    title = "Error in address locations",
    title_x = 0.5,
    xaxis_title = 'Distance between Hex centre & Address (meters)',
    yaxis_title = 'Percentage of hexes (Cumulative)',
    bargap = 0.01,
)

The reverse geocoding is accurate enough. Almost all of the addresses returned are within a few hundred meters of the hexagon centre. In any case, we will mainly be using the hex centres for future functions - the address is mainly for reference.

In [7]:
# Assign to new columns
df_hex['address'] = address
df_hex.to_feather('../data/bangalore_hex_data.feather') # Save file
df_hex.head() # Display final table

<ipython-input-7-3076d0dc17a0>:3: UserWarning:

this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')



,id,ward_no,centre_lat,centre_lon,resolution,geometry,address
0,8860169665fffff,1,13.113131,77.601091,8,"POLYGON ((77.60526 13.11063, 77.60535 13.11561...","Yelahanka, Kempegowda, Yelahanka Zone, Bengalu..."
1,886016975bfffff,1,13.120623,77.597002,8,"POLYGON ((77.60117 13.11812, 77.60126 13.12311...","Canadian International School, 4, Bellary Road..."
2,8860169751fffff,1,13.128092,77.601343,8,"POLYGON ((77.60552 13.12559, 77.60560 13.13058...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo..."
3,8860169669fffff,1,13.098170,77.600839,8,"POLYGON ((77.60501 13.09567, 77.60510 13.10065...","Yelahanka Sante, Shivanahalli Main Road, Gandh..."
4,8860169759fffff,1,13.120601,77.605432,8,"POLYGON ((77.60961 13.11810, 77.60969 13.12308...","Kempegowda, Yelahanka Zone, Bengaluru, Bangalo..."
